In [5]:
import os

import guidance
import requests
from dotenv import load_dotenv


load_dotenv()

True

In [11]:
GOOGLE_CUSTOME_SEARCH_API_KEY=os.getenv("GOOGLE_CUSTOM_SEARCH_API_KEY")
GOOGLE_CUSTOME_SEARCH_ENGINE_ID=os.getenv("CUSTOM_SEARCH_ENGINE_ID")

GOOGLE_CUSTOME_SEARCH_BASE_URL = "https://www.googleapis.com/customsearch/v1"

## Google Search周りを実装

In [12]:
def google_search(search_term: str, num: int = 3) -> dict:
    params = {
        'q': search_term,
        'key': GOOGLE_CUSTOME_SEARCH_API_KEY,
        'cx': GOOGLE_CUSTOME_SEARCH_ENGINE_ID,
        'num': num,
    }
    response = requests.get(GOOGLE_CUSTOME_SEARCH_BASE_URL, params=params)
    return response.json()["items"]


def get_top_snippets(query: str, n: int = 3):
    results = google_search(query, num=n)[:n]
    return [{'title': x['title'], 'snippet': x['snippet']} for x in results]


In [13]:
get_top_snippets("ワンピースの最新刊は今何巻？")

[{'title': 'ONE PIECE（ワンピース）｜漫画最新刊105巻（次は106巻 ...',
  'snippet': 'Mar 3, 2023 ... 最新刊（105巻） · 次巻（106巻） · 全巻まとめセット（1～99巻） · 関連商品 · 最新ニュースランキング · アニメイト通販 · おすすめ特集 · 新着ラジオ番組.'},
 {'title': '漫画「ワンピース」全巻買ったらいくら？まとめ買いの最安値は ...',
  'snippet': '6 days ago ... 結論を言うと、まとめ買いの最安値は電子書籍のクーポンを使うのが一番安いです。Amebaマンガなら単行本の ... 漫画「ワンピース」は何巻まで出てる？'},
 {'title': '【12月スーパーSALE 15%OFF】 ワンピース 83巻〜最新刊105巻 ...',
  'snippet': '2023年最新】ワンピース 最新刊の人気アイテム - メルカリ. 2023年最新】ワンピース 最新刊 ... 新品]ワンピース ONE PIECE (1-105巻 最新刊) 全巻セット :WA-01:漫画.'}]

## guidanceの実装

In [39]:
def is_search(completion: str) -> bool:
    return '<search>' in completion


def search(query: str) -> dict:
    return get_top_snippets(query)



In [40]:
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [74]:
prompt = guidance('''{{#system~}}
あなたは親切なアシスタントです。
{{~/system}}
{{#user~}}
今後、あなたの回答が事実関係に依存する場合は、回答前に<search>query</search>という機能でウェブを検索してください。そうすれば、私がウェブ検索結果を貼り付けますので、あなたはそれに答えることができます。
{{~/user}}
{{#assistant~}}
わかりました、そうさせていただきます。
{{~/assistant}}
{{#user~}}
さぁ始めましょう！
{{~/user}}
{{#assistant~}}
よし、準備OKだ。
{{~/assistant}}
{{#user~}}
{{user_query}}
{{~/user}}
{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}
{{#if (is_search query)}}
{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
{{/if}}''')

In [75]:
query = "今日のマイクロソフトの株価はどうなっているのか？"
p1 = prompt(user_query=query, search=search, is_search=is_search)
p1

Stop program system あなたは親切なアシスタントです。 user 今後、あなたの回答が事実関係に依存する場合は、回答前に<search>query</search>という機能でウェブを検索してください。そうすれば、私がウェブ検索結果を貼り付けますので、あなたはそれに答えることができます。 assistant わかりました、そうさせていただきます。 user さぁ始めましょう！ assistant よし、準備OKだ。 user 今日のマイクロソフトの株価はどうなっているのか？ assistant <search>マイクロソフト 株価 </search> user Search results: 
<result>
 マイクロソフト【MSFT】：株式/株価 - Yahoo!ファイナンス 
 マイクロソフト【MSFT】の株価。NYSE（ニューヨーク証券取引所）とNASDAQに上場している全銘柄の株価やチャート、業績などを網羅。ADR日本株やランキングも充実してい ... 
</result>
<result>
 アルファベット株4％安、韓国サムスンが検索にMSビング採用か ... 
 Apr 17, 2023 ... 韓国サムスン電子 が端末にデフォルトで搭載する検索エンジンとして、グーグルに代わりマイクロソフト の「Bing（ビング）」採用を検討していると ... 
</result>
<result>
 ＜米国株情報＞マイクロソフトの検索エンジン「ビング」が ... 
 Apr 19, 2023 ... ＜米国株情報＞マイクロソフトの検索エンジン「ビング」がグーグルの牙城崩す可能性. 04/19 10:13 配信; ウエルスアドバイザー. マイクロソフト＜MSFT ... 
</result> assistant <search>マイクロソフト 株価</search>の検索結果によると、マイクロソフトの株価はNYSE（ニューヨーク証券取引所）とNASDAQに上場しているため、複数の株価情報があります。最新の株価情報を確認するには、Yahoo!ファイナンスなどの株価情報サイトを参照することができます。ただし、私は株式投資の専門家ではないため、投資に関するアドバイスを提供することはできません。

In [76]:
query = "今日の大阪の天気は？"
p2 = prompt(user_query=query, search=search, is_search=is_search)
p2

Stop program system あなたは親切なアシスタントです。 user 今後、あなたの回答が事実関係に依存する場合は、回答前に<search>query</search>という機能でウェブを検索してください。そうすれば、私がウェブ検索結果を貼り付けますので、あなたはそれに答えることができます。 assistant わかりました、そうさせていただきます。 user さぁ始めましょう！ assistant よし、準備OKだ。 user 今日の大阪の天気は？ assistant <search>大阪の天気 </search> user Search results: 
<result>
 大阪（大阪）の天気 - Yahoo!天気・災害 
 大阪（大阪）の天気予報。今日・明日の天気と風と波、明日までの6時間ごとの降水確率と最高・最低気温を見られます。 
</result>
<result>
 大阪府の天気 - ウェザーニュース 
 大阪府の最新天気情報。よく当たる1時間毎のピンポイント天気、現在の気温や湿度、雨雲レーダー、週間天気が確認できます。都市、施設名、観光名所による検索もこちら ... 
</result>
<result>
 気象庁｜過去の気象データ検索 
 ホーム >; 各種データ・資料 >; 過去の気象データ検索 >; 都府県・地方の選択 >; 地点の選択. 地点の選択. 地点選択用大阪府地図. 大阪府. 地点. カナ. 緯度. 経度. 
</result> assistant 大阪の天気は、<a href="https://weather.yahoo.co.jp/weather/jp/27/6200.html">Yahoo!天気・災害</a>によると、現在は曇りで、最高気温は26℃、最低気温は22℃の予報です。